In [1]:
import geemap
import ee
import os
import pandas as pd

In [2]:
#ee.Authenticate()
#geemap.update_package()

In [3]:
#list of plots in GA LTER dataset (not including broad creek)

ee.Initialize()

Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [16]:
in_csv = 'C:/Users/arj26323/Documents/Data/Biomass datasets/BMA.csv' ##PC
df = geemap.csv_to_pandas(in_csv)
df = df [['Date', 'Longitude', 'Latitude']]

# bma_points = geemap.xy_to_points(in_csv, latitude = 'Latitude', longitude = 'Longitude')
# Map.addLayer(bma_points, {}, 'csv to ee')

Map.add_xy_data(in_csv, x="Longitude", y="Latitude", layer_name="csv to ee", to_ee=True)
fc = geemap.csv_to_ee(in_csv)
Map.addLayer(fc, {}, 'csv to ee 2')

EEException: Invalid JSON payload received. Unexpected token.
: {"constantValue": NaN}, "Elevation": {
                    ^

In [10]:
df
print(df.dtypes)

Date          object
Longitude    float64
Latitude     float64
dtype: object


In [5]:
dean_may = 'E:/BMA planet data/21_May_2021/files/20210521_151835_36_2444_3B_AnalyticMS_SR_clip.tif'
dean_mar_a = 'E:/BMA planet data/22_Mar_2021/files/20210322_154607_103c_3B_AnalyticMS_SR_clip.tif'
dean_mar_b = 'E:/BMA planet data/22_Mar_2021/files/20210322_154608_103c_3B_AnalyticMS_SR_clip.tif'
dean_sep30_a = 'E:/BMA planet data/22_Mar_2021/files/20200930_154455_1011_3B_AnalyticMS_SR_clip.tif'
dean_sep30_b = 'E:/BMA planet data/22_Mar_2021/files/20200930_154456_1011_3B_AnalyticMS_SR_clip.tif'

Map.add_raster(dean_may, colormap='terrain', layer_name='May 2021')

Url()